<a href="https://colab.research.google.com/github/maoo1/meme-propaganda-detector/blob/master/MEME_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[sem eval task](https://propaganda.math.unipd.it/semeval2024task4/teampage.php?passcode=7a4c50dc60f44593a07529d2253593e9)

to do:
- import models
- finetune models
- evaluate models

In [ ]:
!git pull origin master

fatal: not a git repository (or any of the parent directories): .git


In [20]:
!git commit -am "data processing complete, working on finishing up the training loop"

fatal: not a git repository (or any of the parent directories): .git


In [ ]:
!git push origin master


# Setting up libraries and mounting google drive




In [2]:
# install dependencies and codebase
# transformers Hugging Face library: access to BERT / GPT
!pip install torch transformers datasets tqdm gdown==v4.6.3
!pip install evaluate
#!pip install sklearn_hierarchical_classification
!mkdir checkpoints

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
  Attempting uninstall: gdown
    Found existing installation: gdown 5.2.0
    Uninstalling gdown-5.2.0:
      Successfully uninstalled gdown-5.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━

In [3]:
# mounting data and files to drive
# hypothetically can upload SemEval datasets to drive for easier access + we would both have access
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# HIERARCHY

# Preprocessing the Data

In [4]:
import json
from dataclasses import dataclass
from typing import List, Dict, Tuple, Union

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer

creating Meme object and dataset

In [5]:
# represents single meme as text, label
@dataclass
class Meme:
  text: str
  labels: Union[List[str], None] # labels can be none for test data

  @staticmethod
  def from_dict(data: dict):
    text = data["text"]
    labels = data.get("labels")
    if len(labels) == 0:
        labels = ['None']
    return Meme(text=text, labels=labels)

# custom dataset for meme data, supporting multi-label classification
class MemeDataset(Dataset):

  def __init__(self, tokenizer, data: List[Dict], label_encoder):
    MemeDataset.tokenizer = tokenizer
    self.examples = [Meme.from_dict(item) for item in data]
    self.label_encoder = label_encoder

    if label_encoder:
      # encode labels if available

      self.encoded_labels = [label_encoder.fit_transform([item.labels]) for item in self.examples]
    else:
      self.encoded_labels = None

  def __len__(self):
    return len(self.examples)

  # def __getitem__(self, idx):
    # return self.examples[idx]
  def __getitem__(self, idx):
    example = self.examples[idx]
    labels = self.encoded_labels[idx] if self.encoded_labels else None
    return {"text": example.text, "labels": labels}

  # batch processing
  @staticmethod
  def collate_fn(batched_samples: List[Meme], max_length=512):

    batched_texts = [sample.text for sample in batched_samples]
    batched_labels = [sample.labels for sample in batched_samples if sample.labels is not None]

    text_encoding = MemeDataset.tokenizer(
        batched_texts,
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )

    if batched_labels:
      labels_tensor = torch.tensor(batched_labels, dtype=torch.float32) # multi-label as float
    else:
      labels_tensor = None

    return {
        "input_ids": text_encoding["input_ids"],
        "attention_mask": text_encoding["attention_mask"],
        "labels": labels_tensor,
    }

  def inverse_transform_labels(encoded_labels, label_encoder):

    if encoded_labels is not None:
      return label_encoder.inverse_transform(encoded_labels)
    return []

In [6]:
from sklearn.preprocessing import MultiLabelBinarizer
class_labels = ['Appeal to authority', 'Appeal to fear/prejudice', 'Bandwagon',
                'Black-and-white Fallacy/Dictatorship', 'Causal Oversimplification',
                'Doubt', 'Exaggeration/Minimisation', 'Flag-waving',
                'Glittering generalities (Virtue)', 'Loaded Language',
                "Misrepresentation of Someone's Position (Straw Man)",
                'Name calling/Labeling', 'Obfuscation, Intentional vagueness, Confusion',
                'Presenting Irrelevant Data (Red Herring)', 'Reductio ad hitlerum',
                'Repetition', 'Slogans', 'Smears', 'Thought-terminating cliché',
                'Whataboutism', 'None']
mlb = MultiLabelBinarizer(classes=class_labels)

In [7]:
def load_json(file_path):
  with open(file_path, 'r') as f:
    return json.load(f)

# initializing datasets
def initialize_datasets(tokenizer, dataset_paths: Dict, label_encoder) -> dict:

  raw_datasets = {}

  # loading raw datasets from file paths
  for split_name, file_path in dataset_paths.items():
    raw_datasets[split_name] = load_json(file_path)

  # fit mlb with full hierarchy
  #all_labels = list(hierarchy.keys())
  #label_encoder = MultiLabelBinarizer(classes=all_labels)
  #label_encoder.fit([all_labels])
  #print(type(label_encoder))

  # convert raw datasets to MemeDataset objects
  split_datasets = {}
  for split_name, split_data in raw_datasets.items():
    split_datasets[split_name] = MemeDataset(tokenizer, list(split_data), label_encoder)

  return split_datasets

dataset_paths = {
  "train": '/content/drive/MyDrive/NLP_FINAL/train.json',
  "validation": '/content/drive/MyDrive/NLP_FINAL/validation.json',
  # "dev_unlabeled":  '/content/drive/MyDrive/NLP_FINAL/dev_unlabeled.json',
  "test":  '/content/drive/MyDrive/NLP_FINAL/dev_subtask1_en.json'
}

Loading the data

In [17]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

"""
datasets = {}
for split_name in raw_datasets.keys():
  split_data = list(raw_datasets[split_name])

  datasets[split_name] = MemeDataset(tokenizer, split_data)
"""
# loading datasets
datasets = initialize_datasets(tokenizer, dataset_paths, mlb)

length_train = len(datasets['train'])
length_val = len(datasets['validation'])
# length_dev_unlabaled = len(datasets['dev_unlabeled'])
length_test = len(datasets['test'])

print("num training examples:", length_train)
print("num validation examples:", length_val)
print("num test examples:", length_test)

#validation_dataloader = DataLoader(datasets['validation'],
#                                   batch_size=64,
#                                   shuffle=False,
#                                   collate_fn=MemeDataset.collate_fn,
#                                   num_workers=2)

for i in range(0, 6):
  print(datasets['train'][i]['text'])
  print(datasets['train'][i]['labels'])
  print(mlb.inverse_transform(datasets['train'][i]['labels']))

num training examples: 7000
num validation examples: 500
num test examples: 1000
THIS IS WHY YOU NEED\n\nA SHARPIE WITH YOU AT ALL TIMES
[[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
[('Black-and-white Fallacy/Dictatorship',)]
GOOD NEWS!\n\nNAZANIN ZAGHARI-RATCLIFFE AND ANOOSHEH ASHOORI HAVE BEEN RELEASED\n\nAfter years of being unjustly detained in Iran, they are making their way safely back to the UK.
[[0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0]]
[('Glittering generalities (Virtue)', 'Loaded Language')]
PAING PHYO MIN IS FREE!
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]]
[('None',)]
Move your ships away!\n\noooook\n\nMove your ships away!\n\nNo, and I just added 10 more
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]]
[('None',)]
WHEN YOU'RE THE FBI, THEY LET YOU DO IT.
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]]
[('Thought-terminating cliché',)]
PUTIN'S SECRET CAMOUFLAGE ARMY
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]]
[('None',)]


# Checking if Preprocessing Data Correct

In [10]:
def check_label_encoding(dataset, mlb):
    """
    Checks if all items in the dataset have been properly encoded by MLB.
    """
    for i, item in enumerate(dataset.examples):
        if item.labels is not None:
            # Encode labels
            encoded = mlb.transform([item.labels])[0]
            # Convert to 2D numpy array for inverse_transform
            decoded = mlb.inverse_transform(np.array([encoded]))[0]

            # Check that original and decoded labels match
            assert set(item.labels) == set(decoded), \
                f"Mismatch in encoding for sample {i}.\nOriginal: {item.labels}\nDecoded: {decoded}"

    print(f"All labels in {len(dataset.examples)} examples are properly encoded!")

check_label_encoding(datasets['train'], mlb)
check_label_encoding(datasets['validation'], mlb)
check_label_encoding(datasets['test'], mlb)

All labels in 7000 examples are properly encoded!
All labels in 500 examples are properly encoded!
All labels in 1000 examples are properly encoded!


# TRAINING

In [18]:
train_dataloader = DataLoader(datasets['train'],
                               batch_size=64,
                               shuffle=True,
                               collate_fn=MemeDataset.collate_fn,
                               num_workers=2)

validation_dataloader = DataLoader(datasets['validation'],
                                   batch_size=64,
                                   shuffle=False,
                                   collate_fn=MemeDataset.collate_fn,
                                   num_workers=2)
# final eval only
test_dataloader = DataLoader(datasets['test'],
                              batch_size=64,
                              shuffle=False,
                              collate_fn=MemeDataset.collate_fn,
                              num_workers=2)

In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.optim import Optimizer, AdamW
from tqdm.notebook import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

# models bertweet + roberta
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=20, torch_dtype="auto")

bertweet = AutoModelForSequenceClassification.from_pretrained("vinai/bertweet-base")
training_args = TrainingArguments(output_dir="test_trainer", eval_strategy="epoch")

def compute_metrics(eval_pred):

    logits, labels = eval_pred
    # convert logits to binary predictions
    predictions = (logits > 0.5).astype(int)

    # convert predictions and labels to lists of labels for each example
    pred_labels = {
        idx: [mlb.inverse_transform(pred_row)]# for i, pred in enumerate(pred_row) if pred == 1]
        for idx, pred_row in enumerate(predictions)
    }

    #gold_labels = {
    #    idx: [label_map[i] for i, gold in enumerate(label_row) if gold == 1]
    #    for idx, label_row in enumerate(labels)
    #}

    # evaluating hierarchical metrics
    #h_precision, h_recall, h_f1 = evaluate_h(pred_labels, gold_labels)

    #return {
    #    "h_precision": h_precision,
    #    "h_recall": h_recall,
    #    "h_f1": h_f1
    #}

trainer = Trainer(
    model=bertweet,
    args=training_args,
    train_dataset=datasets['train'],
    eval_dataset=datasets['validation'],
    compute_metrics=compute_metrics,
)

trainer.train()